<center>
<img src="https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/afrisenti-logo.png" width="30%" />
</center>

<center>

#SemEval 2023 Shared Task 12: AfriSenti (Task A)

###Starter Notebook

</center>

#Leveraging Pre-trained A Language Model to Train A Sentiment Classifier

**Authors:**
[Idris Abdulmumin](https://www.hausanlp.org/author/idris-abdulmuminu/), [David Adelani](https://dadelani.github.io/) and [Shamsuddeen Hassan Muhammad](https://www.hausanlp.org/author/shamsuddeen-hassan-muhammad/).

**Introduction:** 

You are welcome to participate in our first-of-its-kind SemEval Shared Task! 

In this starter notebook, we will take you through the process of fine-tuning a pre-trained language model on a sample data to build a sentiment classifier. The notebook was adapted from a [Hugginface implementation]( https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_xnli.py) for such tasks.

**Languages (Tracks)**
* Track 1: Hausa 
* Track 2: Yoruba
* Track 3: Igbo
* Track 4: Nigerian_Pidgin
* Track 5: Amharic
* Track 6: Algerian Arabic
* Track 7: Moroccan Arabic/Darija,
* Track 8: Swahili
* Track 9: Kinyarwanda (data to be released soon)
* Track 10: Twi (data to be released soon)
* Track 11: Mozambican Portuguese
* Track 12: Setswana (data to be released soon)
* Track 13: isiZulu (data to be released soon)

**Level:** <font color='blue'>`Beginner to Intermediate`</font>

**Outline:** 

1. Installation and importation of necessary libraries
2. Setting up the project parameters.
3. Running training and evaluation

**Before you start:**

It is **strongly advised** that you use a GPU to speed up training. To do this, go to the "Runtime" menu in Colab, select "Change runtime type" and then in the popup menu, choose "GPU" in the "Hardware accelerator" box.

NB: 

- **The codes in this notebook are provided to familiarize yourselves with fine-tuning language models for sentiment classification. You may extend and (or) modify as appropriate to obtain competitive performances**

- **We also use the data as it is, without any cleaning such as removal of emoji and hyperlinks.**




#1) Installations and imports

##a. Mount drive (if you are running on colab)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##b. Clone or update competition repository

After cloning, under MyDrive, you will see afrisenti-semeval-2023 folder with all the the data for the afrisenti shared task (training and dev) 

In [ ]:
%cd /content/drive/MyDrive

import os

PROJECT_DIR = '/content/drive/MyDrive/afrisent-semeval-2023'
PROJECT_GITHUB_URL = 'https://github.com/afrisenti-semeval/afrisent-semeval-2023.git'

if not os.path.isdir(PROJECT_DIR):
  !git clone {PROJECT_GITHUB_URL}
else:
  %cd {PROJECT_DIR}
  !git pull {PROJECT_GITHUB_URL}

/content/drive/MyDrive
/content/drive/MyDrive/afrisent-semeval-2023
From https://github.com/afrisenti-semeval/afrisent-semeval-2023
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


##c. Install required libraries

- Set the project dire
ctory in the cell below, where the requirements file should also be located, and run the cell

In [ ]:
if os.path.isdir(PROJECT_DIR):
  #The requirements file should be in PROJECT_DIR
  if os.path.isfile(os.path.join(PROJECT_DIR, 'starter_kit/requirements.txt')):
    !pip install -r starter_kit/requirements.txt
  else:
    print('requirements.txt file not found')

else:
  print("Project directory not found, please check again.")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 30.8 MB/s 
     |████████████████████████████████| 148 kB 67.8 MB/s 
     |████████████████████████████████| 1.3 MB 61.7 MB/s 
     |████████████████████████████████| 441 kB 65.2 MB/s 
     |████████████████████████████████| 72 kB 1.5 MB/s 
     |████████████████████████████████| 115 kB 69.5 MB/s 
     |████████████████████████████████| 212 kB 67.3 MB/s 
     |████████████████████████████████| 163 kB 74.1 MB/s 
     |████████████████████████████████| 95 kB 6.1 MB/s 
     |████████████████████████████████| 127 kB 70.0 MB/s 
     |████████████████████████████████| 7.6 MB 60.7 MB/s 
     |████████████████████████████████| 115 kB 70.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing in

##d. Import libraries

Import libraries below

In [ ]:
import pandas as pd
import numpy as np

# Please don not edit anything here
languages = ['am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo']

#2) Dataset

##a. Formatting

The training dataset that was provided for the competition is in the following format:

| ID | text | label |
| --- | --- | --- |
| twt001 | example text | negative |
| twt002 | example text | positive |
| ... | ... | ... |

However, the code in the starter kit do not expect the 
ID and require the training (and evaluation) data to be in the following format

|text | label |
|--- | --- |
|example text | negative |
|example text | positive |
|... | ... |

To reformat the data run the following cell

In [ ]:
# Training Data Paths

TASK = 'SubtaskA'
TRAINING_DATA_DIR = os.path.join(PROJECT_DIR, TASK, 'train')
FORMATTED_TRAIN_DATA = os.path.join(TRAINING_DATA_DIR, 'formatted-train-data')

if os.path.isdir(TRAINING_DATA_DIR):
  print('Data directory found.')
  if not os.path.isdir(FORMATTED_TRAIN_DATA):
    print('Creating directory to store formatted data.')
    os.mkdir(FORMATTED_TRAIN_DATA)
else:
  print(TRAINING_DATA_DIR + ' is not a valid directory or does not exist!')

Data directory found.


In [ ]:
%cd {TRAINING_DATA_DIR}

training_files = os.listdir()

if len(training_files) > 0:
  for training_file in training_files:
    if training_file.endswith('.tsv'):

      data = training_file.split('_')[0]
      if not os.path.isdir(os.path.join(FORMATTED_TRAIN_DATA, data)):
        print(data, 'Creating directory to store train, dev and test splits.')
        os.mkdir(os.path.join(FORMATTED_TRAIN_DATA, data))
      
      df = pd.read_csv(training_file, sep='\t', names=['ID', 'text', 'label'], header=0)
      df[['text', 'label']].to_csv(os.path.join(FORMATTED_TRAIN_DATA, data, 'train.tsv'), sep='\t', index=False)
    else:
      print(training_file + ' skipped!')
else:
  print('No files are found in this directory!')

/content/drive/MyDrive/afrisent-semeval-2023/SubtaskA/train
README.txt skipped!
formatted-train-data skipped!
splitted-train-dev-test skipped!
splitted-train-dev skipped!


After running the code above, a new folder (called formated-train-data) with formated files is created in the "datasets" folder in the train sub-folder.

##b. <font color='red'>`(Optional) Creating Evaluation (Dev and Test) sets from the available training data`</font>

You may wish to create train and evaluation (dev and test) sets from the training data provided. If you wish to do so, you can run any of the cells below`

###i. If you want to create both the Dev and Test sets, run this cell

In [ ]:
if os.path.isdir(FORMATTED_TRAIN_DATA):
  print('Data directory found.')
  SPLITTED_DATA = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev-test')
  if not os.path.isdir(SPLITTED_DATA):
    print('Creating directory to store train, dev and test splits.')
    os.mkdir(SPLITTED_DATA)
else:
  print(FORMATTED_TRAIN_DATA + ' is not a valid directory or does not exist!')

%cd {FORMATTED_TRAIN_DATA}
formatted_training_files = os.listdir()

if len(formatted_training_files) > 0:
  for data_name in formatted_training_files:
    formatted_training_file = os.path.join(data_name, 'train.tsv')
    if os.path.isfile(formatted_training_file):
      labeled_tweets = pd.read_csv(formatted_training_file, sep='\t', names=['text', 'label'], header=0)
      train, dev, test = np.split(labeled_tweets.sample(frac=1, random_state=42), [int(.7*len(labeled_tweets)), int(.8*len(labeled_tweets))])

      if not os.path.isdir(os.path.join(SPLITTED_DATA, data_name)):
        print(data_name, 'Creating directory to store train, dev and test splits.')
        os.mkdir(os.path.join(SPLITTED_DATA, data_name))

      train.sample(frac=1).to_csv(os.path.join(SPLITTED_DATA, data_name, 'train.tsv'), sep='\t', index=False)
      dev.sample(frac=1).to_csv(os.path.join(SPLITTED_DATA, data_name, 'dev.tsv'), sep='\t', index=False)
      test.sample(frac=1).to_csv(os.path.join(SPLITTED_DATA, data_name,'test.tsv'), sep='\t', index=False)
    else:
      print(training_file + ' is not a supported file!')
else:
  print('No files are found in this directory!')

Data directory found.
Creating directory to store train, dev and test splits.
/content/drive/MyDrive/afrisent-semeval-2023/SubtaskA/train/formatted-train-data
am Creating directory to store train, dev and test splits.
dz Creating directory to store train, dev and test splits.
ha Creating directory to store train, dev and test splits.
ig Creating directory to store train, dev and test splits.
ma Creating directory to store train, dev and test splits.
pcm Creating directory to store train, dev and test splits.
pt Creating directory to store train, dev and test splits.
sw Creating directory to store train, dev and test splits.
yo Creating directory to store train, dev and test splits.


After running the code above, a new folder (called splitted-train-dev-test) with train-dev-test split is created in the "datasets" folder in the train sub-folder. Here, the train-dev-test split is 70/10/20



###ii. If you want to create only the Dev set from the training data, please run this

In [ ]:
from sklearn.model_selection import train_test_split

if os.path.isdir(FORMATTED_TRAIN_DATA):
  print('Data directory found.')
  SPLITTED_DATA = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev')
  if not os.path.isdir(SPLITTED_DATA):
    print('Creating directory to store train, dev and test splits.')
    os.mkdir(SPLITTED_DATA)
else:
  print(FORMATTED_TRAIN_DATA + ' is not a valid directory or does not exist!')

%cd {FORMATTED_TRAIN_DATA}
formatted_training_files = os.listdir()

if len(formatted_training_files) > 0:
  for data_name in formatted_training_files:
    formatted_training_file = os.path.join(data_name, 'train.tsv')
    if os.path.isfile(formatted_training_file):
      labeled_tweets = pd.read_csv(formatted_training_file, sep='\t', names=['text', 'label'], header=0)
      train, dev = train_test_split(labeled_tweets, test_size=0.3)

      if not os.path.isdir(os.path.join(SPLITTED_DATA, data_name)):
        print(data_name, 'Creating directory to store train, dev and test splits.')
        os.mkdir(os.path.join(SPLITTED_DATA, data_name))

      train.sample(frac=1).to_csv(os.path.join(SPLITTED_DATA, data_name, 'train.tsv'), sep='\t', index=False)
      dev.sample(frac=1).to_csv(os.path.join(SPLITTED_DATA, data_name, 'dev.tsv'), sep='\t', index=False)
    else:
      print(training_file + ' is not a supported file!')
else:
  print('No files are found in this directory!')

Data directory found.
Creating directory to store train, dev and test splits.
/content/drive/MyDrive/afrisent-semeval-2023/SubtaskA/train/formatted-train-data
am Creating directory to store train, dev and test splits.
dz Creating directory to store train, dev and test splits.
ha Creating directory to store train, dev and test splits.
ig Creating directory to store train, dev and test splits.
ma Creating directory to store train, dev and test splits.
pcm Creating directory to store train, dev and test splits.
pt Creating directory to store train, dev and test splits.
sw Creating directory to store train, dev and test splits.
yo Creating directory to store train, dev and test splits.


After running the code above, a new folder (called splitted-train-dev) with train-dev split is created in the "datasets" folder in the train sub-folder. Here, the train-dev split is 70/30


#3) Training setup

##a. Set project parameters

For a list of models that be used for fine-tuning, you can check [HERE](https://huggingface.co/models).

### i. set language code

In [ ]:
# Language to train sentiment classifier for
LANGUAGE_CODE = 'ha' # 'am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo'

In [ ]:
# Language to train sentiment classifier for
LANGUAGE_CODE = 'am' # 'am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo'

In [ ]:
# Language to train sentiment classifier for
LANGUAGE_CODE = 'dz' # 'am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo'

In [ ]:
# Language to train sentiment classifier for
LANGUAGE_CODE = 'ig' # 'am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo'

In [ ]:
# Language to train sentiment classifier for
LANGUAGE_CODE = 'ma' # 'am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo'

In [ ]:
# Language to train sentiment classifier for
LANGUAGE_CODE = 'pcm' # 'am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo'

In [ ]:
# Language to train sentiment classifier for
LANGUAGE_CODE = 'pt' # 'am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo'

In [ ]:
# Language to train sentiment classifier for
LANGUAGE_CODE = 'sw' # 'am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo'

In [ ]:
# Language to train sentiment classifier for
LANGUAGE_CODE = 'yo' # 'am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo'

### ii. set other params

In [ ]:
%cd {PROJECT_DIR}

# # Language to train sentiment classifier for
# LANGUAGE_CODE = 'am' # 'am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo' DO NOT EASILTY UNCOMMENT THIS LINE

if LANGUAGE_CODE in languages:
  # Model Training Parameters
  MODEL_NAME_OR_PATH = 'Davlan/afro-xlmr-mini'
  BATCH_SIZE = 32
  LEARNING_RATE = 5e-5 #0.005
  NUMBER_OF_TRAINING_EPOCHS = 5 #100
  MAXIMUM_SEQUENCE_LENGTH = 128
  SAVE_STEPS = -1

  print('Everything set. You can now start model training.')

else:
  print("Invalid language code/Dataset not released. Please input any of the following released data\n\n\t- 'am'\n\t- 'dz'\n\t- 'ha'\n\t- 'ig'\n\t- 'ma'\n\t- 'pcm'\n\t- 'pt'\n\t- 'sw'\n\t- 'yo'")

/content/drive/MyDrive/afrisent-semeval-2023
Everything set. You can now start model training.


##b. Train the model

In the section below, we provide three options: 

- 1) training model without any validation; 
- 2) training model with validation but without testing; 
- 3) training a model with validation and test set.

###i. Training on only Train set, without any evaluation

In [ ]:
DATA_DIR = os.path.join(TRAINING_DATA_DIR, 'formatted-train-data', LANGUAGE_CODE)
OUTPUT_DIR = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE + '_no_eval')

!CUDA_VISIBLE_DEVICES=0 python starter_kit/run_textclass.py \
  --model_name_or_path {MODEL_NAME_OR_PATH} \
  --data_dir {DATA_DIR} \
  --do_train \
  --per_device_train_batch_size {BATCH_SIZE} \
  --learning_rate {LEARNING_RATE} \
  --num_train_epochs {NUMBER_OF_TRAINING_EPOCHS} \
  --max_seq_length {MAXIMUM_SEQUENCE_LENGTH} \
  --output_dir {OUTPUT_DIR} \
  --save_steps {SAVE_STEPS}

As you may observe, the training loss is very large. As a start, you can tune the training parameters and model to get a competitive result. 

You can observe also, there is no validation metrics (e.g., accuracy, loss etc) since we are only training without validtaion 

###ii. Training on only Train and Dev sets

In [ ]:
DATA_DIR = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev', LANGUAGE_CODE)
OUTPUT_DIR = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE + '_no_test')

!CUDA_VISIBLE_DEVICES=0 python starter_kit/run_textclass.py \
  --model_name_or_path {MODEL_NAME_OR_PATH} \
  --data_dir {DATA_DIR} \
  --do_train \
  --do_eval \
  --per_device_train_batch_size {BATCH_SIZE} \
  --learning_rate {LEARNING_RATE} \
  --num_train_epochs {NUMBER_OF_TRAINING_EPOCHS} \
  --max_seq_length {MAXIMUM_SEQUENCE_LENGTH} \
  --output_dir {OUTPUT_DIR} \
  --save_steps {SAVE_STEPS}

Now, you can observe, there is evalidation metrics (e.g., accuracy, loss etc) since we are evaluating our model performance on the validation set we created from the 
training data.






###iii. Training with Train, Dev and Test sets

In [ ]:
DATA_DIR = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev-test', LANGUAGE_CODE)
OUTPUT_DIR = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE)

!CUDA_VISIBLE_DEVICES=0 python starter_kit/run_textclass.py \
  --model_name_or_path {MODEL_NAME_OR_PATH} \
  --data_dir {DATA_DIR} \
  --do_train \
  --do_eval \
  --do_predict \
  --per_device_train_batch_size {BATCH_SIZE} \
  --learning_rate {LEARNING_RATE} \
  --num_train_epochs {NUMBER_OF_TRAINING_EPOCHS} \
  --max_seq_length {MAXIMUM_SEQUENCE_LENGTH} \
  --output_dir {OUTPUT_DIR} \
  --save_steps {SAVE_STEPS}

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=True,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_met

Now that you trained your best model and find the best  parameters, you can submit your prediction on dev or test set on CodaLab competition page.

#4) Submission

- For submission after training your model, unlabeled tweets were provided for dev (development phase) and test (evaluation phase). 

- To generate their sentiment prediction, provide the path to the file containing the unlabeled tweets.

**What the code does**
1. Predicting sentiments of the unlabeled tweets (dev or test)
2. Create a file in the submission format

In [ ]:
%cd {PROJECT_DIR}

OUTPUT_DIR = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE)
FILE_NAME = os.path.join(PROJECT_DIR, TASK, 'dev', LANGUAGE_CODE + '_dev.tsv')
TEXT_COLUMN = 'tweet'

!python starter_kit/run_predict.py \
  --model_path {OUTPUT_DIR} \
  --file_name {FILE_NAME} \
  --text_column {TEXT_COLUMN} \
  --lang_code {LANGUAGE_CODE}

/content/drive/MyDrive/afrisent-semeval-2023
***** Running Prediction *****
  Num examples = 2835
  Batch size = 8
100% 355/355 [00:07<00:00, 49.02it/s]
Data directory found.
Creating submission files directory.


- Congratulations. You now trained sentiment classifier and predict on the unlabelled tweets.

- The prediction file (pred_{lang}.tsv) is in "afrisenti-semval-2023" folder under "submissions" folder. For example, since we trained hausa (ha), you will be able to see pred_ha.tsv file ready for submission. The submission file is in the format below:

<center>

|ID | label |
|--- | --- |
|hau_dev_00001| negative |
|hau_dev_00002| positive |
|... | ... |

</center>

- Inside the same folder, you will also see a file "ha_predictions.tsv" with the format below to see tweets with corresponding sentiment predictions. This file is not for submission.


<center>

|ID | text | label |
|--- | --- | --- | 
|hau_dev_00001| @user Allah Miki albarkah 🙏🙏🙏 |  positive |
|hau_dev_00002| @user Kidan ma zai dadi😂	 |  negative |
|... | ... | ... |

</center>






# 5) Evaluate on test subsets from train set -- p/r/f1 scores

## a. predict and create a scores folder

In [ ]:
# copied from previous cells, just to make the later cells in this section 5) work

import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# ------- UNCOMMENT IF NEEDED ---------------
# if os.path.isdir(PROJECT_DIR):
#   #The requirements file should be in PROJECT_DIR
#   if os.path.isfile(os.path.join(PROJECT_DIR, 'starter_kit/requirements.txt')):
#     !pip install -r starter_kit/requirements.txt
#   else:
#     print('requirements.txt file not found')

# else:
#   print("Project directory not found, please check again.")
# -------------------------------------------

TASK = 'SubtaskA'

# ------- UNCOMMENT IF NEEDED ---------------
# Language to train sentiment classifier for
# LANGUAGE_CODE = 'am'
# -------------------------------------------

if LANGUAGE_CODE in languages:
  # Model Training Parameters
  MODEL_NAME_OR_PATH = 'Davlan/afro-xlmr-mini'
  BATCH_SIZE = 32
  LEARNING_RATE = 5e-5 #0.005
  NUMBER_OF_TRAINING_EPOCHS = 5 #100
  MAXIMUM_SEQUENCE_LENGTH = 128
  SAVE_STEPS = -1

  print('Everything set. You can now start model training.')

else:
  print("Invalid language code/Dataset not released. Please input any of the following released data\n\n\t- 'am'\n\t- 'dz'\n\t- 'ha'\n\t- 'ig'\n\t- 'ma'\n\t- 'pcm'\n\t- 'pt'\n\t- 'sw'\n\t- 'yo'")

# will create the resulting files in a folder called 'test-set-scores'
%cd {PROJECT_DIR}

OUTPUT_DIR = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE)
FILE_NAME = os.path.join(PROJECT_DIR, TASK, 'train', 'splitted-train-dev-test', LANGUAGE_CODE, 'test.tsv')
TEXT_COLUMN = 'text'

!python starter_kit/afrisenti_get_scores1.py \
  --model_path {OUTPUT_DIR} \
  --file_name {FILE_NAME} \
  --text_column {TEXT_COLUMN} \
  --lang_code {LANGUAGE_CODE}

print('current language code is:', LANGUAGE_CODE)
print('current model path is:', OUTPUT_DIR)

Everything set. You can now start model training.
/content/drive/MyDrive/afrisent-semeval-2023
***** Running Prediction *****
  Num examples = 1705
  Batch size = 8
100% 214/214 [00:06<00:00, 33.56it/s]
Data directory found.
Creating scores files directory.
current language code is: yo
current model path is: /content/drive/MyDrive/afrisent-semeval-2023/models/yo


## b. evaluation method

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

def evaluate(file_true, file_pred):

  df_true = pd.read_csv(file_true, sep='\t')
  df_pred = pd.read_csv(file_pred, sep='\t')
  true = df_true.label
  pred = df_pred.label
  return classification_report(true, pred)

## c. get scores for 'ha'

In [ ]:
# get scores for 'ha'
LANGUAGE_CODE = 'ha'
file_true = os.path.join(PROJECT_DIR, TASK, 'train', 'splitted-train-dev-test', LANGUAGE_CODE, 'test.tsv')
file_pred = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE, 'scores', 'pred_ha.tsv')
print(evaluate(file_true, file_pred))

              precision    recall  f1-score   support

    negative       0.67      0.64      0.65       889
     neutral       0.69      0.70      0.69       980
    positive       0.82      0.84      0.83       966

    accuracy                           0.73      2835
   macro avg       0.73      0.73      0.73      2835
weighted avg       0.73      0.73      0.73      2835



## d. get scores for 'am'

In [ ]:
# get scores for 'am'
LANGUAGE_CODE = 'am'
file_true = os.path.join(PROJECT_DIR, TASK, 'train', 'splitted-train-dev-test', LANGUAGE_CODE, 'test.tsv')
file_pred = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE, 'scores', 'pred_' + LANGUAGE_CODE + '.tsv')
print(evaluate(file_true, file_pred))

              precision    recall  f1-score   support

    negative       0.43      0.28      0.34       275
     neutral       0.63      0.77      0.69       639
    positive       0.64      0.53      0.58       283

    accuracy                           0.60      1197
   macro avg       0.57      0.53      0.54      1197
weighted avg       0.59      0.60      0.59      1197



## e. get scores for 'dz'

In [ ]:
# get scores for 'dz'
LANGUAGE_CODE = 'dz'
file_true = os.path.join(PROJECT_DIR, TASK, 'train', 'splitted-train-dev-test', LANGUAGE_CODE, 'test.tsv')
file_pred = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE, 'scores', 'pred_' + LANGUAGE_CODE + '.tsv')
print(evaluate(file_true, file_pred))

              precision    recall  f1-score   support

    negative       0.50      1.00      0.67       167
     neutral       0.00      0.00      0.00        62
    positive       0.00      0.00      0.00       102

    accuracy                           0.50       331
   macro avg       0.17      0.33      0.22       331
weighted avg       0.25      0.50      0.34       331



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## f. get scores for 'ig'

In [ ]:
# get scores for 'ig'
LANGUAGE_CODE = 'ig'
file_true = os.path.join(PROJECT_DIR, TASK, 'train', 'splitted-train-dev-test', LANGUAGE_CODE, 'test.tsv')
file_pred = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE, 'scores', 'pred_' + LANGUAGE_CODE + '.tsv')
print(evaluate(file_true, file_pred))

              precision    recall  f1-score   support

    negative       0.66      0.60      0.63       507
     neutral       0.74      0.75      0.74       906
    positive       0.76      0.79      0.77       626

    accuracy                           0.73      2039
   macro avg       0.72      0.71      0.71      2039
weighted avg       0.72      0.73      0.72      2039



##g. get scores for 'ma'

In [ ]:
# get scores for 'ma'
LANGUAGE_CODE = 'ma'
file_true = os.path.join(PROJECT_DIR, TASK, 'train', 'splitted-train-dev-test', LANGUAGE_CODE, 'test.tsv')
file_pred = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE, 'scores', 'pred_' + LANGUAGE_CODE + '.tsv')
print(evaluate(file_true, file_pred))

              precision    recall  f1-score   support

    negative       0.68      0.63      0.66       318
     neutral       0.74      0.84      0.79       443
    positive       0.72      0.65      0.68       356

    accuracy                           0.72      1117
   macro avg       0.71      0.71      0.71      1117
weighted avg       0.72      0.72      0.72      1117



## h. get scores for 'pcm'

In [ ]:
# get scores for 'pcm'
LANGUAGE_CODE = 'pcm'
file_true = os.path.join(PROJECT_DIR, TASK, 'train', 'splitted-train-dev-test', LANGUAGE_CODE, 'test.tsv')
file_pred = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE, 'scores', 'pred_' + LANGUAGE_CODE + '.tsv')
print(evaluate(file_true, file_pred))

              precision    recall  f1-score   support

    negative       0.73      0.85      0.79       641
     neutral       0.00      0.00      0.00        20
    positive       0.64      0.49      0.56       364

    accuracy                           0.71      1025
   macro avg       0.46      0.45      0.45      1025
weighted avg       0.68      0.71      0.69      1025



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## i. get scores for 'pt'

In [ ]:
# get scores for 'pt'
LANGUAGE_CODE = 'pt'
file_true = os.path.join(PROJECT_DIR, TASK, 'train', 'splitted-train-dev-test', LANGUAGE_CODE, 'test.tsv')
file_pred = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE, 'scores', 'pred_' + LANGUAGE_CODE + '.tsv')
print(evaluate(file_true, file_pred))

              precision    recall  f1-score   support

    negative       0.44      0.20      0.28       144
     neutral       0.54      0.93      0.69       320
    positive       0.00      0.00      0.00       149

    accuracy                           0.53       613
   macro avg       0.33      0.38      0.32       613
weighted avg       0.39      0.53      0.42       613



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## j. get scores for 'sw'

In [ ]:
# get scores for 'sw'
LANGUAGE_CODE = 'sw'
file_true = os.path.join(PROJECT_DIR, TASK, 'train', 'splitted-train-dev-test', LANGUAGE_CODE, 'test.tsv')
file_pred = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE, 'scores', 'pred_' + LANGUAGE_CODE + '.tsv')
print(evaluate(file_true, file_pred))

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        33
     neutral       0.58      0.94      0.72       204
    positive       0.59      0.15      0.24       125

    accuracy                           0.58       362
   macro avg       0.39      0.36      0.32       362
weighted avg       0.53      0.58      0.49       362



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## k. get scores for 'yo'

In [ ]:
# get scores for 'yo'
LANGUAGE_CODE = 'yo'
file_true = os.path.join(PROJECT_DIR, TASK, 'train', 'splitted-train-dev-test', LANGUAGE_CODE, 'test.tsv')
file_pred = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE, 'scores', 'pred_' + LANGUAGE_CODE + '.tsv')
print(evaluate(file_true, file_pred))

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00       354
     neutral       0.00      0.00      0.00       644
    positive       0.41      1.00      0.59       707

    accuracy                           0.41      1705
   macro avg       0.14      0.33      0.20      1705
weighted avg       0.17      0.41      0.24      1705



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
